In [ ]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [ ]:
sub = re.compile("[A-Za-z]+")
text_words = set(word for word in nltk.word_tokenize(text.lower()) if sub.search(word))
ind_to_word = dict(enumerate(text_words))
word_to_ind = {j:i for i,j in ind_to_word.items()}
word_to_ind

{'libtorch': 0,
 'and': 1,
 'version': 2,
 'available': 3,
 'pytorch': 4,
 'tested': 5,
 'currently': 6,
 'of': 7,
 'supported': 8,
 'stable': 9,
 'note': 10,
 'install': 11,
 'preferences': 12,
 'run': 13,
 'only': 14,
 'the': 15,
 'command': 16,
 'c++': 17,
 'is': 18,
 'that': 19,
 'most': 20,
 'for': 21,
 'represents': 22,
 'select': 23,
 'your': 24}

In [ ]:
fs = 'Select your preferences and run the install command'
fs = fs.lower()
fs = nltk.word_tokenize(fs)
fs

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [ ]:
fs_vect = torch.zeros(len(fs))
for i,word in enumerate(fs):
    fs_vect[i] = word_to_ind[word]
fs_vect

tensor([23., 24., 12.,  1., 13., 15., 11., 16.])

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [ ]:
fs_matrx = torch.zeros((len(fs), len(word_to_ind)))
for i, word in enumerate(fs):
    fs_matrx[i, word_to_ind[word]] = 1
fs_matrx

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.]])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [ ]:
fs_vect_ = fs_vect.type(torch.long)
fs_vect_

tensor([23, 24, 12,  1, 13, 15, 11, 16])

In [ ]:
nn.Embedding(num_embeddings=len(text_words), embedding_dim=len(text_words))(fs_vect_)

tensor([[-0.1631,  0.1824, -1.6145, -2.0389, -0.5137,  1.0800,  0.7620, -1.2372,
         -0.0063,  0.3325, -0.2095,  0.3245, -0.5928, -1.0001, -1.5142, -0.1946,
          0.0201, -1.1963,  1.6309, -1.0088,  0.7910, -0.3188,  0.5548,  0.3527,
         -0.5171],
        [ 0.3349, -1.1374, -0.2967,  0.1038,  0.2872,  1.6163,  0.8835, -0.8971,
         -1.1376,  1.3834, -1.1561,  0.2274,  0.0929,  1.1636, -0.2336,  1.5451,
          0.8763,  0.2755, -0.8199, -0.9402,  2.1048,  0.1306,  0.5448,  0.3013,
         -0.8456],
        [ 0.0999,  0.3531, -0.7670, -0.4885, -0.8026, -1.5330,  0.2634, -0.5073,
         -1.5415,  0.8513,  1.2109, -1.1880, -0.4185,  0.5132, -0.1557, -1.0545,
          0.3440, -1.3580,  0.2491, -0.4964,  0.2829, -0.1350, -0.9876, -1.1049,
          0.4160],
        [ 0.2587, -1.8525,  1.7193, -0.3955, -0.3431, -0.7139,  0.1381, -0.0085,
          0.5217,  0.9449, -0.1012, -0.1081, -1.4514, -0.7364,  1.3831, -0.1073,
          1.1250, -1.3109, -0.2625,  0.6078, -1.1839

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`.

In [ ]:
df = pd.read_csv("./data/surnames.csv")

2.2 Закодировать национальности числами, начиная с 0.

In [ ]:
class_encoder = LabelEncoder()
df['nationality'] = class_encoder.fit_transform(df['nationality'])

2.3 Разбить датасет на обучающую и тестовую выборку

In [ ]:
X = df['surname'].str.lower().str.strip()
y = df['nationality']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

In [ ]:
class Vocab:
    def __init__(self, data):
        self.max_seq_len = data.str.len().max()
        tokens = set()
        for item in data:
            tokens.update(item)
        tokens = list(tokens)
        tokens.insert(0, "<PAD>")
        self.idx_to_token = dict(enumerate(tokens))
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины


In [ ]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, surname):
        '''Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2)'''
        surname_t = torch.zeros(self.vocab.max_seq_len).type(torch.long)
        for i, token in enumerate(surname):
            surname_t[i] = self.vocab.token_to_idx[token]
        return surname_t

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта:
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


In [ ]:
def hard_training(model, criterion, optimizer, n_epochs=51):
    for epoch in range(n_epochs):
        epoch_loss = 0
        y_pred = torch.empty(0)
        y_true = torch.empty(0)

        model.train()
        for X_batch, y_batch in train_loader:
            predictions = model.forward(X_batch, training=True)
            loss = criterion(predictions, y_batch.type(torch.long))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item()
            with torch.no_grad():
                y_true = torch.cat((y_true, y_batch))
                y_pred = torch.cat((y_pred, predictions.argmax(dim=1).cpu().detach()))
        trin_acc = accuracy_score(y_true, y_pred).item()


        model.eval()
        with torch.no_grad():
            val_loss = 0
            y_pred = torch.empty(0)
            y_true = torch.empty(0)
            for X_batch, y_batch in test_loader:
                predictions = model(X_batch)
                loss = criterion(predictions, y_batch.type(torch.long)).item()
                y_true = torch.cat((y_true, y_batch))
                y_pred = torch.cat((y_pred, predictions.argmax(dim=1).cpu().detach()))
                val_loss += loss
            val_acc = accuracy_score(y_true, y_pred).item()
            if epoch % 5 == 0:
                print(f'#{epoch} Training loss: {epoch_loss / len(train_loader):.4f}\
 training_acc: {trin_acc:.4f} val_loss: {val_loss / len(test_loader):.4f} val_acc: {val_acc:.4f}')

def GloP_boB(model):
    y_pred = torch.empty(0)
    y_ = torch.empty(0)

    for X_batch, y_batch in test_loader:

        predictions = model(X_batch).argmax(dim=1).cpu().detach()
        y_pred = torch.cat((y_pred, predictions))
        y_ = torch.cat((y_, y_batch))

    print(classification_report(y_, y_pred, zero_division=0))

In [ ]:
vocab = Vocab(X)
train_dataset = SurnamesDataset(X_train, y_train, vocab)
test_dataset = SurnamesDataset(X_test, y_test, vocab)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
n_classes

18

In [ ]:
class MyOwnClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(MyOwnClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.l1 = nn.Sequential(
            nn.Conv1d(embedding_dim, 64, kernel_size=5),
            nn.BatchNorm1d(64),
            nn.MaxPool1d(2),
        )

        self.fc1 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(6*64, 1024),
            nn.ReLU(),

        )
        self.fc2 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(1024, 18),
            nn.Softmax(dim=1),
        )

    def forward(self, x, training=False):
        x = self.embedding(x).transpose(1,2)
        x = self.l1(x)
#         x = self.l2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = MyOwnClassifier(num_embeddings=vocab.vocab_len, embedding_dim=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
sum(p.numel() for p in model.parameters())

413258

In [ ]:
hard_training(model, criterion, optimizer)

NameError: name 'hard_training' is not defined

In [ ]:
GloP_boB(model)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       346
         1.0       0.00      0.00      0.00        36
         2.0       0.00      0.00      0.00        81
         3.0       0.00      0.00      0.00        49
         4.0       0.31      0.82      0.45       567
         5.0       0.00      0.00      0.00        36
         6.0       0.00      0.00      0.00       118
         7.0       0.00      0.00      0.00        32
         8.0       0.00      0.00      0.00        41
         9.0       0.00      0.00      0.00       108
        10.0       0.00      0.00      0.00       161
        11.0       0.00      0.00      0.00        15
        12.0       0.00      0.00      0.00        25
        13.0       0.00      0.00      0.00        14
        14.0       0.51      0.71      0.59       482
        15.0       0.00      0.00      0.00        13
        16.0       0.00      0.00      0.00        57
        17.0       0.00    

In [ ]:
model2 = MyOwnClassifier(num_embeddings=vocab.vocab_len, embedding_dim=32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

In [ ]:
hard_training(model2, criterion, optimizer, n_epochs=101)

#0 Training loss: 2.5730 training_acc: 0.4130 val_loss: 2.4885 val_acc: 0.4923
#5 Training loss: 2.4303 training_acc: 0.5515 val_loss: 2.4282 val_acc: 0.5492
#10 Training loss: 2.4144 training_acc: 0.5666 val_loss: 2.4228 val_acc: 0.5551
#15 Training loss: 2.3763 training_acc: 0.6056 val_loss: 2.3815 val_acc: 0.5984
#20 Training loss: 2.3502 training_acc: 0.6314 val_loss: 2.3693 val_acc: 0.6129
#25 Training loss: 2.3439 training_acc: 0.6375 val_loss: 2.3584 val_acc: 0.6198
#30 Training loss: 2.3147 training_acc: 0.6669 val_loss: 2.3265 val_acc: 0.6489
#35 Training loss: 2.3039 training_acc: 0.6778 val_loss: 2.3206 val_acc: 0.6603
#40 Training loss: 2.2925 training_acc: 0.6891 val_loss: 2.3228 val_acc: 0.6562
#45 Training loss: 2.2905 training_acc: 0.6902 val_loss: 2.3171 val_acc: 0.6621
#50 Training loss: 2.2871 training_acc: 0.6942 val_loss: 2.3108 val_acc: 0.6667
#55 Training loss: 2.2832 training_acc: 0.6977 val_loss: 2.3088 val_acc: 0.6712
#60 Training loss: 2.2801 training_acc: 0.

In [ ]:
GloP_boB(model2)

              precision    recall  f1-score   support

         0.0       0.89      0.96      0.92       346
         1.0       0.45      0.86      0.59        36
         2.0       0.00      0.00      0.00        81
         3.0       0.00      0.00      0.00        49
         4.0       0.58      0.92      0.71       567
         5.0       0.00      0.00      0.00        36
         6.0       0.00      0.00      0.00       118
         7.0       0.00      0.00      0.00        32
         8.0       0.00      0.00      0.00        41
         9.0       0.53      0.74      0.62       108
        10.0       0.75      0.86      0.80       161
        11.0       0.33      0.07      0.11        15
        12.0       0.00      0.00      0.00        25
        13.0       0.00      0.00      0.00        14
        14.0       0.77      0.81      0.79       482
        15.0       0.00      0.00      0.00        13
        16.0       0.00      0.00      0.00        57
        17.0       0.00    

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [ ]:
def test_surname(surname):
    x = train_dataset.vectorize(surname).unsqueeze(0)
    variety, predictions = model(x).topk(k=3, dim=1)
    variety = variety.cpu().detach().squeeze()
    pred_ = class_encoder.inverse_transform(predictions.cpu().detach().squeeze())
    out_ = ", ".join([f"{nat}:{frac:.2f}" for nat, frac in zip(pred_, variety)])
    print(f"{surname} --- {out_}")

In [ ]:
for surname in ("bazoyan", "gorshenev", "strizhov", "petukhov", "trubadurov", "atayanc"):
    test_surname(surname)

bazoyan --- Russian:1.00, English:0.00, Italian:0.00
gorshenev --- Russian:1.00, English:0.00, Arabic:0.00
strizhov --- Russian:1.00, English:0.00, Italian:0.00
petukhov --- English:1.00, Russian:0.00, Arabic:0.00
trubadurov --- Russian:1.00, English:0.00, German:0.00
atayanc --- English:1.00, Arabic:0.00, Czech:0.00


## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

In [ ]:
positive = pd.read_csv("./data/positive_reviews.txt", sep='%-%', header=None, engine="python")
positive["type"] = "positive"
negative = pd.read_csv("./data/negative_reviews.txt", sep='%-%', header=None, engine="python")
negative["type"] = "negative"
df = pd.concat((positive, negative), ignore_index=True)
df.columns = ["review", "type"]
df.head(2)

,review,type
0,"simplistic , silly and tedious .",positive
1,"it's so laddish and juvenile , only teenage bo...",positive


In [ ]:
class_encoder_2 = LabelEncoder()
df.type = class_encoder_2.fit_transform(df.type)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
puttern = re.compile("^[a-z]+$")
def preprocess(text):
    text = text.lower().strip()
    words = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text) if (puttern.search(word)) and (word not in stopwords)]
    return words

In [ ]:
df.review = df.review.apply(preprocess)

In [ ]:
X = df.review
y = df.type
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
class Vocab:
    def __init__(self, data):
        self.max_seq_len = data.apply(lambda x: len(x)).max()
        tokens = set()
        for item in data:
            tokens.update(item)
        tokens = list(tokens)
        tokens.insert(0, "<PAD>")
        tokens.insert(1, "<UNK>")
        self.idx_to_token = dict(enumerate(tokens))
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, review):
        '''Генерирует представление отзыва'''
        review_t = torch.zeros(self.vocab.max_seq_len).type(torch.long)
        for i, token in enumerate(review):
            try:
                review_t[i] = self.vocab.token_to_idx[token]
            except KeyError as ke:
                review_t[i] = self.vocab.token_to_idx["<UNK>"]
        return review_t

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`
    - подберите адекватную размерность вектора эмбеддинга:
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [ ]:
vocab = Vocab(X_train)
train_dataset = ReviewsDataset(X_train, y_train, vocab)
test_dataset = ReviewsDataset(X_test, y_test, vocab)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
class MyOwnClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, dropout_level=0.5):
        super(MyOwnClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.l1 = nn.Sequential(
            nn.Conv1d(embedding_dim, 256, kernel_size=5),
            nn.BatchNorm1d(256),
            nn.MaxPool1d(2),
        )
        self.l2 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=3),
            nn.BatchNorm1d(512),
            nn.MaxPool1d(2),
        )
#         self.l3 = nn.Sequential(
#             nn.Conv1d(256, 512, kernel_size=3),
#             nn.BatchNorm1d(512),
#             nn.MaxPool1d(2),
#         )
        self.drop = nn.Dropout(p=dropout_level)
        self.fc1 = nn.Sequential(
            nn.Linear(6*512, 1024),
#             nn.Softmax(dim=1),
            nn.ReLU(),

        )
#         self.fc2 = nn.Sequential(
#             nn.Dropout(p=dropout_level),
#             nn.Linear(1024, 256),
# #             nn.Softmax(dim=1),
#             nn.ReLU(),
#         )
        self.fc3 = nn.Sequential(
            nn.Linear(1024, n_classes),
            nn.Softmax(dim=1),
        )

    def forward(self, x, training=False):
#         print(training)
        x = self.embedding(x).transpose(1,2)
        x = self.l1(x)
        x = self.l2(x)
#         x = self.l3(x)
        x = x.view(x.size(0), -1)
        if training:
            x = self.drop(x)
        x = self.fc1(x)
#         x = self.fc2(x)
        if training:
            x = self.drop(x)
        x = self.fc3(x)
        return x

In [ ]:
model = MyOwnClassifier(num_embeddings=vocab.vocab_len, embedding_dim=128, dropout_level=0.9)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
hard_training(model, criterion, optimizer, n_epochs=101)

#0 Training loss: 0.7655 training_acc: 0.4960 val_loss: 0.7087 val_acc: 0.5035
#5 Training loss: 0.7448 training_acc: 0.5135 val_loss: 0.6946 val_acc: 0.5148
#10 Training loss: 0.7278 training_acc: 0.5277 val_loss: 0.6932 val_acc: 0.5546
#15 Training loss: 0.7233 training_acc: 0.5226 val_loss: 0.6897 val_acc: 0.5649
#20 Training loss: 0.6777 training_acc: 0.5854 val_loss: 0.6734 val_acc: 0.5917
#25 Training loss: 0.6434 training_acc: 0.6386 val_loss: 0.6786 val_acc: 0.5748
#30 Training loss: 0.5983 training_acc: 0.6887 val_loss: 0.6653 val_acc: 0.6071
#35 Training loss: 0.5670 training_acc: 0.7295 val_loss: 0.6536 val_acc: 0.6329
#40 Training loss: 0.5124 training_acc: 0.7901 val_loss: 0.6581 val_acc: 0.6273
#45 Training loss: 0.4705 training_acc: 0.8362 val_loss: 0.6984 val_acc: 0.5963
#50 Training loss: 0.4417 training_acc: 0.8674 val_loss: 0.6595 val_acc: 0.6324
#55 Training loss: 0.4138 training_acc: 0.8968 val_loss: 0.6948 val_acc: 0.6015
#60 Training loss: 0.3900 training_acc: 0.

In [ ]:
GloP_boB(model)

              precision    recall  f1-score   support

         0.0       0.62      0.72      0.67      1067
         1.0       0.67      0.56      0.61      1066

    accuracy                           0.64      2133
   macro avg       0.65      0.64      0.64      2133
weighted avg       0.65      0.64      0.64      2133



2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [ ]:
def test_review(review):
    x = train_dataset.vectorize(review).unsqueeze(0)
    variety, predictions = model(x).topk(k=1, dim=1)
    variety = variety.cpu().detach().view(-1)
    pred_ = predictions.cpu().detach().view(-1)
    pred_ = class_encoder_2.inverse_transform(pred_)
    out_ = ", ".join([f"{nat}:{frac:.2f}" for nat, frac in zip(pred_, variety)])
    print(f"{review} --- {out_}")

In [ ]:
reviews_ = [
    "The last time I had lunch here, I really liked the soup. Excellent. I'll come again.",
    "It feels like I dined on frogs. I will not visit this institution again."
]

In [ ]:
for review in reviews_:
     test_review(preprocess(review))

['last', 'time', 'lunch', 'really', 'liked', 'soup', 'excellent', 'come'] --- positive:0.60
['feel', 'like', 'dined', 'frog', 'visit', 'institution'] --- negative:1.00
